In [1]:
from googleapiclient.discovery import build
import pandas as pd
pd.set_option('display.max_rows', 500)
api_key = 'ENTER API KEY' 
youtube = build('youtube', 'v3', developerKey=api_key)

In [2]:
year_list = ['2022', '2023']
segmentdf = pd.read_excel('VehicleSegments.xlsx')
segmentdf.drop('Unnamed: 0', axis = 1, inplace = True)

In [4]:
#Remove EVs to Optimize Youtube Queries
#Remove duplicated Grand Cherokee from Midsize SUV
seg2 = segmentdf.drop(segmentdf[(segmentdf['Model'] == 'Ariya') | (segmentdf['Model'] == 'Leaf') | (segmentdf['Model'] == 'C-hr') | (segmentdf['Model'] == 'Bz4x') | (segmentdf['Model'] == 'Bolt') | (segmentdf['Model'] == 'Ioniq') | (segmentdf['Model'] == 'Ioniq 5') | (segmentdf['Model'] == 'Ev6') | (segmentdf['Model'] == 'Nexo') | (segmentdf['Model'] == 'Ram') | (segmentdf['Model'] == 'R1t launch') | (segmentdf['Model'] == 'Endurance') | (segmentdf['Model'] == 'Solterra') | (segmentdf['Model'] == 'Mustang mach-e') | (segmentdf['Model'] == 'Hummer') | (segmentdf['Model'] == 'Id4')].index)
seg2 = seg2.drop(seg2[seg2['Model'] == 'Grand cherokee'].index[0])
seg2 = seg2.reset_index()
seg2.drop('index', axis = 1, inplace = True)

In [5]:
seg2["Make"].value_counts()

Nissan        14
Toyota        13
Chevrolet     11
Ford          10
Hyundai        9
Honda          8
Jeep           8
Gmc            8
Kia            8
Volkswagen     7
Subaru         6
Mazda          6
Buick          4
Porsche        3
Audi           2
Mitsubishi     2
Bmw            1
Fiat           1
Dodge          1
Acura          1
Name: Make, dtype: int64

In [6]:
#Youtube API per account cannot handle all models to be scraped at once. So they need to be segregated as shown below. 
#Safest way is to do it one at a time and then switch api key to different accounts.
#Use below code to systematically generate files as per make and then merge them in the end. 
segloc = seg2.loc[(seg2['Make'] == "Ford") | (seg2['Make'] == "Honda")]

In [7]:
query_list = []
for i in year_list:
    for j in segloc["Model"]:
        query = segloc.loc[segloc["Model"] == j]["Make"].tolist()[0] + ' ' + j + ' ' + str(i)
        query_list.append(query)

In [8]:
query_list

['Honda Civic 2022',
 'Honda Insight 2022',
 'Honda Accord 2022',
 'Ford Ecosport 2022',
 'Honda Hr-v 2022',
 'Ford Bronco 2022',
 'Ford Edge 2022',
 'Honda Passport 2022',
 'Ford Explorer 2022',
 'Honda Pilot 2022',
 'Ford Expedition 2022',
 'Ford Maverick 2022',
 'Ford Ranger 2022',
 'Honda Ridgeline 2022',
 'Ford F series 2022',
 'Ford Bronco sport 2022',
 'Ford Escape 2022',
 'Honda Cr-v 2022',
 'Honda Civic 2023',
 'Honda Insight 2023',
 'Honda Accord 2023',
 'Ford Ecosport 2023',
 'Honda Hr-v 2023',
 'Ford Bronco 2023',
 'Ford Edge 2023',
 'Honda Passport 2023',
 'Ford Explorer 2023',
 'Honda Pilot 2023',
 'Ford Expedition 2023',
 'Ford Maverick 2023',
 'Ford Ranger 2023',
 'Honda Ridgeline 2023',
 'Ford F series 2023',
 'Ford Bronco sport 2023',
 'Ford Escape 2023',
 'Honda Cr-v 2023']

In [9]:
len(query_list)

36

In [10]:
#Can be used to get the list of URLs to be scraped - Choose ones with maximum comments to maintain efficiency of queries. 
#But this process is incredibly intensive on the quota and has to be used sparingly. 
#Hence a combination of this and manual process of searching and grabbing URLs directly from youtube were used. 
#Scaling this requires permission from Google which can be done by 
#the organization if required through increasing Quota limits substantially. 10000 points per day is inadequate. 
#Only 2022 and 2023 will be considered here due to query limits
urls = []
for i in query_list:
    search_response = youtube.search().list(q=i, part='id,snippet', maxResults=5).execute()
    video_ids = [search_result['id']['videoId'] for search_result in search_response.get('items', [])]
    video_response = youtube.videos().list(id=','.join(video_ids), part='id,snippet,statistics').execute()
    try:
        for video_result in video_response.get('items', []):
            title = video_result['snippet']['title']
            url = f'https://www.youtube.com/watch?v={video_result["id"]}'
            video_id = video_result["id"]
            view_count = video_result['statistics']['viewCount']
            comment_count = video_result['statistics']['commentCount']
            urls.append((i, title, url, video_id, view_count, comment_count))
    except:
        print(i)
        continue
yturl_df = pd.DataFrame(urls, columns = ['Query', 'Title', 'URL', 'VideoID', 'ViewCount', 'Comment_Count'])

In [11]:
yturl_df['Comment_Count'] = yturl_df['Comment_Count'].astype(int)
#Taking only the top result by comments. Due to a large number of vehicles over 2 years, this needs to be done to stay within query limits
yturl_df_top = yturl_df.groupby('Query').apply(lambda x: x.nlargest(5, 'Comment_Count')).reset_index(drop=True)
#Removing duplicate results - Sometimes same results come up as top results for similar queries - Unpopular vehicles for both year queries for instance
#Needs to be done to avoid duplicating scraping. 
yturl_df_top = yturl_df_top.drop(yturl_df_top[yturl_df_top.drop('Query', axis = 1).duplicated()].index)

In [12]:
yturl_df_top

,Query,Title,URL,VideoID,ViewCount,Comment_Count
0,Ford Bronco 2022,2022 Ford Bronco Raptor - interior Exterior De...,https://www.youtube.com/watch?v=zH3zaUvwcv4,zH3zaUvwcv4,1944390,414
1,Ford Bronco 2022,Is the 2022 Ford Wildtrak the Bronco trim that...,https://www.youtube.com/watch?v=0k2YT6dsrp4,0k2YT6dsrp4,107153,353
2,Ford Bronco 2022,2022 Ford Bronco Raptor - POV Driving Impressions,https://www.youtube.com/watch?v=t_5Zx2IB48Q,t_5Zx2IB48Q,407351,340
3,Ford Bronco 2022,2022 Ford Bronco Sport // The Affordable Bronc...,https://www.youtube.com/watch?v=DWvQN5_EqC0,DWvQN5_EqC0,258516,236
4,Ford Bronco 2022,2022 Ford Bronco review // Eats huge bumps for...,https://www.youtube.com/watch?v=iVcgYBhT3E4,iVcgYBhT3E4,75597,162
5,Ford Bronco 2023,Is the 2023 Ford Bronco Raptor the best NEW pe...,https://www.youtube.com/watch?v=LlDG_-ilagg,LlDG_-ilagg,81836,322
6,Ford Bronco 2023,6 Reasons to Buy a 2023 Ford Bronco (What's New?),https://www.youtube.com/watch?v=8V46N_2almY,8V46N_2almY,151407,299
7,Ford Bronco 2023,OFF-ROAD 4x4 Test 2023 Ford BRONCO,https://www.youtube.com/watch?v=qK20jhZShSE,qK20jhZShSE,169556,114
8,Ford Bronco 2023,2023 Ford Bronco Wildtrak (HOSS 3.0 Fox Shocks...,https://www.youtube.com/watch?v=7Zwf-H3LTmo,7Zwf-H3LTmo,45970,58
9,Ford Bronco 2023,"2023 Ford Bronco Badlands Manual: Start Up, Te...",https://www.youtube.com/watch?v=A5ANU7C7TgA,A5ANU7C7TgA,1535,15


In [13]:
replies = []
comments = []
#commcount = 0
for vid in yturl_df_top['VideoID']:
    video_response=youtube.commentThreads().list(part='snippet,replies',videoId=vid).execute()
    model = yturl_df_top.loc[yturl_df_top["VideoID"] == vid]["Query"].tolist()[0]
    #for q, w in video_ids_dict.items():
    #    if vid in [p for p in w]:
    #        model = q
    url = 'https://www.youtube.com/watch?v='+vid
    title = yturl_df_top.loc[yturl_df_top["VideoID"] == vid]["Title"].tolist()[0]
    #title_response = youtube.videos().list(part='snippet',id=vid).execute()
    #title = title_response['items'][0]['snippet']['title']
    while video_response:
        #extraction based on nested dictionary structure of the video_response object
        for item in video_response['items']:
            #Comment Extraction
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            #Replies of a comment
            replycount = item['snippet']['totalReplyCount']
            try:
                if replycount>0:
                    for reply in item['replies']['comments']:
                    # Extract reply
                        reply = reply['snippet']['textDisplay']
                        replies.append(reply)
            except:
                print('Reply Err')
 
            #Display Comments and Corresponding Replies
            if replies == list():
                replies = "None"
            #print(commcount)
            comments.append((model, comment, replies, title, url))
            #commcount = commcount + 1
            #print(comment)
            replies = []
 
            #"Pagination"/Show More 
        if 'nextPageToken' in video_response:
            video_response = youtube.commentThreads().list(part = 'snippet,replies',videoId = vid, 
                                                           pageToken = video_response['nextPageToken']).execute()
        else:
            break

In [14]:
df = pd.DataFrame(comments, columns = ['Model', 'Comments', 'Replies', 'Video Title', 'URL'])

In [15]:
df.to_csv('YoutubeComments.csv')

In [109]:
df = pd.read_csv('YoutubeComments.csv')
df.drop('Unnamed: 0', axis = 1, inplace = True)
df['Make'] = df['Model'].str.split().str.get(0)
df['Year'] = df['Model'].str.split().str.get(2)
df['Model'] = df['Model'].str.split().str.get(1)
df2 = pd.read_excel('VehicleSegments.xlsx')
df2.drop('Unnamed: 0', axis = 1, inplace = True)
df_full = pd.merge(df, df2, on = ["Make", "Model"])
df_full = df_full[['Segment', 'Make', 'Model', 'Year', 'Comments', 'Replies', 'Video Title', 'URL']]
df_full['Word Count'] = df_full['Comments'].str.split(' ').str.len()

In [110]:
df_full.to_csv('YoutubeCommentsFull.csv')

In [111]:
df_full.head()

,Segment,Make,Model,Year,Comments,Replies,Video Title,URL,Word Count
0,Compact,Hyundai,Elantra,2022,They stepped up there game,None,The 2022 Hyundai Elantra N Is a Surprisingly S...,https://www.youtube.com/watch?v=nn2-7jMkSGs,5
1,Compact,Hyundai,Elantra,2022,the way he says N HAHAHAHAHAHAHAH you can feel...,None,The 2022 Hyundai Elantra N Is a Surprisingly S...,https://www.youtube.com/watch?v=nn2-7jMkSGs,11
2,Compact,Hyundai,Elantra,2022,I like how no one accepts the fact that the el...,['@A Dude That Play&#39;s Games Ifkr they shar...,The 2022 Hyundai Elantra N Is a Surprisingly S...,https://www.youtube.com/watch?v=nn2-7jMkSGs,42
3,Compact,Hyundai,Elantra,2022,Nothing makes me happier than the disappointed...,None,The 2022 Hyundai Elantra N Is a Surprisingly S...,https://www.youtube.com/watch?v=nn2-7jMkSGs,17
4,Compact,Hyundai,Elantra,2022,I need help Hyundai Elantra n or dodge charger...,None,The 2022 Hyundai Elantra N Is a Surprisingly S...,https://www.youtube.com/watch?v=nn2-7jMkSGs,10


In [116]:
dfe = pd.read_csv('Edmunds_CustomerReviews.csv', encoding = "utf-16")

In [126]:
l = dfe['Review'].str.split().str.len().tolist()
avg_edmunds_reviewlength = sum(l)/len(l)

In [199]:
df_final = df_full.loc[df_full["Word Count"] > 40].reset_index()
df_final.drop('index', axis = 1, inplace = True)
df_final = df_final.drop_duplicates()
df_final.to_csv('YoutubeComments_40_InfoRich.csv')

In [189]:
df_final['Make'].value_counts()

Nissan        4592
Toyota        4342
Mazda         2588
Honda         2578
Chevrolet     2062
Ford          1793
Kia           1630
Hyundai       1589
Volkswagen    1193
Gmc           1140
Subaru        1057
Acura          652
Porsche        557
Jeep           528
Audi           285
Mitsubishi     278
Buick          151
Dodge           90
Bmw             38
Fiat            22
Name: Make, dtype: int64